In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/2021-ai-w7-p1/mnist_test.csv
/kaggle/input/2021-ai-w7-p1/mnist_train.csv
/kaggle/input/2021-ai-w7-p1/submission.csv


In [2]:
import torch
import torch.optim as optim

In [3]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
torch.manual_seed(777)

import random
random.seed(777)
if device == 'cuda':
    torch.cuda.manual_seed_all(777)

In [4]:
LR=0.001
epochs=15
BS=100
drop=0.3

In [5]:
train=pd.read_csv('/kaggle/input/2021-ai-w7-p1/mnist_train.csv')
test=pd.read_csv('/kaggle/input/2021-ai-w7-p1/mnist_test.csv')
submit=pd.read_csv('/kaggle/input/2021-ai-w7-p1/submission.csv')
print(train.shape)
print(test.shape)
print(submit.head())

(60000, 786)
(10000, 785)
   id  Label
0   0      1
1   1      1
2   2      1
3   3      1
4   4      1


In [6]:
x=train.iloc[:,1:-1]
y=train.iloc[:,-1]
xtest=test.iloc[:,1:]
print(x.max().max())
print(xtest.max().max())

255.0
255.0


In [7]:
x/=255
xtest/=255
print(x.max().max())
print(xtest.max().max())

1.0
1.0


In [8]:
x=torch.FloatTensor(x.values).to(device)
y=torch.LongTensor(y.values).to(device)
xtest=torch.FloatTensor(xtest.values).to(device)
print(x.shape)
print(y.shape)
print(xtest.shape)

torch.Size([60000, 784])
torch.Size([60000])
torch.Size([10000, 784])


In [9]:
dataset=torch.utils.data.TensorDataset(x,y)
DataLoader=torch.utils.data.DataLoader(dataset,batch_size=BS,shuffle=True)

In [10]:
linear1=torch.nn.Linear(784,512,bias=True)
linear2=torch.nn.Linear(512,512,bias=True)
linear3=torch.nn.Linear(512,512,bias=True)
linear4=torch.nn.Linear(512,512,bias=True)
linear5=torch.nn.Linear(512,10,bias=True)
relu=torch.nn.ReLU()
dropout=torch.nn.Dropout(p=drop)

In [11]:
torch.nn.init.xavier_uniform_(linear1.weight)
torch.nn.init.xavier_uniform_(linear2.weight)
torch.nn.init.xavier_uniform_(linear3.weight)
torch.nn.init.xavier_uniform_(linear4.weight)
torch.nn.init.xavier_uniform_(linear5.weight)

Parameter containing:
tensor([[-0.0565,  0.0423, -0.0155,  ...,  0.1012,  0.0459, -0.0191],
        [ 0.0772,  0.0452, -0.0638,  ...,  0.0476, -0.0638,  0.0528],
        [ 0.0311, -0.1023, -0.0701,  ...,  0.0412, -0.1004,  0.0738],
        ...,
        [ 0.0334,  0.0187, -0.1021,  ...,  0.0280, -0.0583, -0.1018],
        [-0.0506, -0.0939, -0.0467,  ..., -0.0554, -0.0325,  0.0640],
        [-0.0183, -0.0123,  0.1025,  ..., -0.0214,  0.0220, -0.0741]],
       requires_grad=True)

In [12]:
model=torch.nn.Sequential(
    linear1,relu,dropout,
    linear2,relu,dropout,
    linear3,relu,dropout,
    linear4,relu,dropout,
    linear5).to(device)
loss=torch.nn.CrossEntropyLoss().to(device)
optimizer=torch.optim.Adam(model.parameters(),lr=LR)

In [13]:
le=len(DataLoader)
model.train()

for epoch in range(epochs):
    total_cost=0
    for X, Y in DataLoader:
        optimizer.zero_grad()
        cost=loss(model(X),Y)
        cost.backward()
        optimizer.step()
        total_cost+=cost/le
    print(epoch,'cost {:.9f}'.format(total_cost))

0 cost 0.307737172
1 cost 0.145624638
2 cost 0.112738363
3 cost 0.094752125
4 cost 0.081102587
5 cost 0.076060072
6 cost 0.068984285
7 cost 0.066080607
8 cost 0.056281734
9 cost 0.057654969
10 cost 0.053252812
11 cost 0.049256250
12 cost 0.045293149
13 cost 0.050689258
14 cost 0.041668333


In [14]:
with torch.no_grad():
    model.eval()
    prediction=model(x)
    correct=torch.argmax(prediction,dim=1)==y
    print((correct.float().sum()/len(correct)).item())

0.9955000281333923


In [15]:
with torch.no_grad():
    model.eval()
    test_prediction=torch.argmax(model(xtest),dim=1)
    print(test_prediction)

tensor([7, 2, 1,  ..., 4, 5, 6], device='cuda:0')


In [16]:
submit['Label']=test_prediction.cpu().numpy()
print(submit.head())

   id  Label
0   0      7
1   1      2
2   2      1
3   3      0
4   4      4


In [17]:
submit.to_csv('submission.csv',index=False,mode='w')